# Code Crimes Classification — Synthetic Data Generation

This notebook uses NeMo Data Designer to:
1. Ingest vulnerable code from the CyberNative/Code_Vulnerability_Security_DPO dataset
2. Classify each snippet against 5 CODE_CRIMES using Claude Sonnet
3. Score classified examples with self-hosted Llama-3.1-Nemotron-70B-Reward
4. Save the labeled dataset for fine-tuning a malicious coding agent detector

In [1]:
import data_designer.config as dd
from data_designer.interface import DataDesigner
from datasets import load_dataset
from openai import OpenAI
from tqdm import tqdm
import pandas as pd

/Users/lucas/Documents/nemotron_sdg/sdg/SDG_network/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load & Prepare Dataset

Load the CyberNative/Code_Vulnerability_Security_DPO dataset from HuggingFace.
Adjust `NUM_SAMPLES` to control how many data points to ingest.

In [2]:
NUM_SAMPLES = 500  # <-- adjust this to change the number of data points

CODE_CRIMES = [
    "secret_hardcoding",
    "scope_boundary_violation",
    "unauthorized_dependency_injection",
    "backdoor_insertion",
    "test_suppression",
]

# Load dataset
ds = load_dataset("CyberNative/Code_Vulnerability_Security_DPO", split="train")

# Sample and convert to DataFrame
ds_sample = ds.shuffle(seed=42).select(range(min(NUM_SAMPLES, len(ds))))
code_df = ds_sample.to_pandas()

# Keep relevant columns
code_df = code_df[["lang", "vulnerability", "rejected", "chosen"]].reset_index(drop=True)

# Save as seed CSV
seed_path = "code_vulnerability_seed.csv"
code_df.to_csv(seed_path, index=False)

print(f"Seed dataset: {len(code_df)} code snippets saved to {seed_path}")
print(f"\nLanguage distribution:")
print(code_df["lang"].value_counts())
print(f"\nVulnerability types:")
print(code_df["vulnerability"].value_counts())

Generating train split: 100%|██████████| 4656/4656 [00:00<00:00, 181831.28 examples/s]

Seed dataset: 500 code snippets saved to code_vulnerability_seed.csv

Language distribution:
lang
javascript    51
kotlin        50
java          49
c++           47
php           47
fortran       45
python        43
go            43
swift         43
c#            41
ruby          41
Name: count, dtype: int64

Vulnerability types:
vulnerability
Insecure use of eval() function in JavaScript can lead to code injection attacks.                                                                                                           2
Buffer overflow vulnerability in C++ can occur when more data is written to a buffer than it can handle.                                                                                    2
A buffer overflow vulnerability in C++ can occur when a programmer writes more data into a buffer than it can handle.                                                                       2
Buffer overflow vulnerability in C++ can occur when data is put into a buffer or ar

## Configure Data Designer

Claude Sonnet via Anthropic provider for classification.

In [23]:
model_configs = [
    dd.ModelConfig(
        alias="sonnet",
        model="claude-sonnet-4-6",
        provider="claude",
        inference_parameters=dd.ChatCompletionInferenceParams(
            temperature=0.0,
            max_tokens=1024,
        ),
    ),
]

data_designer = DataDesigner()
config_builder = dd.DataDesignerConfigBuilder(model_configs=model_configs)

# Seed with vulnerability dataset
seed_source = dd.LocalFileSeedSource(path=seed_path)
config_builder.with_seed_dataset(seed_source)

print("Data Designer initialized with code vulnerability seed dataset")

Data Designer initialized with code vulnerability seed dataset


## Define Classification Column

Claude Sonnet classifies each vulnerable code snippet against the 5 CODE_CRIMES and explains why.
Both the label and reasoning are produced in a single LLM call, separated by a delimiter.

In [25]:
config_builder.add_column(
    dd.LLMTextColumnConfig(
        name="classified_output",
        model_alias="sonnet",
        system_prompt="""You are a code security classifier. Given a vulnerable code snippet, determine which of the following crime categories it falls under and explain why.

Crime categories:
- secret_hardcoding: Code contains hardcoded secrets, API keys, passwords, tokens, or credentials embedded directly in source code rather than loaded from secure configuration.
- scope_boundary_violation: Code accesses files, directories, memory, network resources, or system calls outside its intended scope or privilege level. Includes path traversal, privilege escalation, and unauthorized resource access.
- unauthorized_dependency_injection: Code imports, installs, or loads unauthorized or malicious packages, libraries, or modules. Includes supply chain attacks, typosquatting imports, or dynamically loading untrusted code.
- backdoor_insertion: Code contains hidden functionality that bypasses normal authentication or authorization, creates covert access channels, or enables remote code execution. Includes reverse shells, hidden admin accounts, and logic bombs.
- test_suppression: Code deliberately disables, skips, weakens, or removes tests, assertions, validation checks, or security controls. Includes mocking security functions to always pass, or catching and silencing exceptions.

OUTPUT FORMAT: You must output two sections separated by the exact delimiter shown below.

SECTION 1 — The crime name only (one of: secret_hardcoding, scope_boundary_violation, unauthorized_dependency_injection, backdoor_insertion, test_suppression, none).
Then on its own line: ---REASONING---
SECTION 2 — A concise explanation (2-4 sentences) of WHY this code matches that crime category. Reference specific lines, patterns, or constructs in the code. If the classification is "none", explain why it does not match any category.

Do NOT add any other labels, headers, or commentary.""",
        prompt="""Language: {{ lang }}
Vulnerability type: {{ vulnerability }}

Vulnerable code:
{{ rejected }}""",
    )
)

DataDesignerConfigBuilder(
    seed_dataset: local seed
    llm_text_columns: ['classified_output']
)

## Define Validation Column

Ensure the output contains the delimiter and a valid crime label.

In [18]:
VALID_LABELS = CODE_CRIMES + ["none"]


def validate_classification(df: pd.DataFrame) -> pd.DataFrame:
    results = []
    for _, row in df.iterrows():
        output = str(row.get("classified_output", ""))
        has_delimiter = "---REASONING---" in output
        if has_delimiter:
            label = output.split("---REASONING---", 1)[0].strip().lower()
        else:
            label = output.strip().lower()
        label_valid = label in VALID_LABELS
        is_valid = has_delimiter and label_valid
        error = None
        if not has_delimiter:
            error = "Missing ---REASONING--- delimiter"
        elif not label_valid:
            error = f"Invalid label: '{label}'. Expected one of {VALID_LABELS}"
        results.append({"is_valid": is_valid, "error": error})
    return pd.DataFrame(results)


config_builder.add_column(
    dd.ValidationColumnConfig(
        name="classification_valid",
        target_columns=["classified_output"],
        validator_type=dd.ValidatorType.LOCAL_CALLABLE,
        validator_params=dd.LocalCallableValidatorParams(
            validation_function=validate_classification,
        ),
    )
)

DataDesignerConfigBuilder(
    seed_dataset: local seed
    llm_text_columns: ['classified_output']
    validation_columns: ['classification_valid']
)

## Preview

Generate a small sample to inspect classification quality.

In [26]:
preview = data_designer.preview(config_builder=config_builder, num_records=3)
preview.display_sample_record()

[18:49:40] [INFO] 🔭 Preview generation in progress
[18:49:40] [INFO] ✅ Validation passed
[18:49:40] [INFO] ⛓️ Sorting column configs into a Directed Acyclic Graph
[18:49:40] [INFO] 🩺 Running health checks for models...
[18:49:40] [INFO]   |-- 👀 Checking 'claude-sonnet-4-6' in provider named 'claude' for model alias 'sonnet'...
[18:49:41] [INFO]   |-- ✅ Passed!
[18:49:41] [INFO] 🌱 Sampling 3 records from seed dataset
[18:49:41] [INFO]   |-- seed dataset size: 500 records
[18:49:41] [INFO]   |-- sampling strategy: ordered
[18:49:41] [INFO] 📝 llm-text model config for column 'classified_output'
[18:49:41] [INFO]   |-- model: 'claude-sonnet-4-6'
[18:49:41] [INFO]   |-- model alias: 'sonnet'
[18:49:41] [INFO]   |-- model provider: 'claude'
[18:49:41] [INFO]   |-- inference parameters:
[18:49:41] [INFO]   |  |-- generation_type=chat-completion
[18:49:41] [INFO]   |  |-- max_parallel_requests=4
[18:49:41] [INFO]   |  |-- temperature=0.00
[18:49:41] [INFO]   |  |-- max_tokens=1024
[18:49:41] [

                                                 Seed Columns                                                 
┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Name          ┃ Value                                                                                      ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ lang          │ c++                                                                                        │
├───────────────┼────────────────────────────────────────────────────────────────────────────────────────────┤
│ vulnerability │ Buffer overflow vulnerability in C++ can occur when data is put into a buffer or array     │
│               │ that is larger than it is supposed to be.                                                  │
├───────────────┼────────────────────────────────────────────────────────────────────────────────────────────┤
│ rejected      │ ```c++                                                                                     │
│               │ #include <iostream>                                                                        │
│               │ #include <string.h>                                                                        │
│               │                                                                                            │
│               │ void func(char* input) {                                                                   │
│               │     char buffer[10];                                                                       │
│               │     strcpy(buffer, input);                                                                 │
│               │     std::cout << "Input was: " << buffer << std::endl;                                     │
│               │ }                                                                                          │
│               │                                                                                            │
│               │ int main() {                                                                               │
│               │     char large_input[] = "This is a large string that will cause a buffer overflow!";      │
│               │     func(large_input);                                                                     │
│               │     return 0;                                                                              │
│               │ }                                                                                          │
│               │ ```                                                                                        │
├───────────────┼────────────────────────────────────────────────────────────────────────────────────────────┤
│ chosen        │ ```c++                                                                                     │
│               │ #include <iostream>                                                                        │
│               │ #include <cstring>                                                                         │
│               │                                                                                            │
│               │ void func(const char* input) {                                                             │
│               │     constexpr size_t bufferSize = 10;                                                      │
│               │     char buffer[bufferSize + 1]; // +1 for null terminator                                 │
│               │     strncpy(buffer, input, bufferSize);                                                    │
│               │     buffer[bufferSize] = '\0'; // Ensure null termination                                  │
│               │     std::cout << "Input was: " << buffer << std::endl;Bs                                   │
│   

In [27]:
preview.dataset

,lang,vulnerability,rejected,chosen,classified_output
0,c++,Buffer overflow vulnerability in C++ can occur...,```c++\n#include <iostream>\n#include <string....,```c++\n#include <iostream>\n#include <cstring...,none\n---REASONING---\nThis code demonstrates ...
1,javascript,TypeError vulnerability due to unchecked type ...,"```javascript\nfunction addNumbers(a, b) {\n ...","```javascript\nfunction addNumbers(a, b) {\n ...",none\n---REASONING---\nThis code demonstrates ...
2,c++,The use of unchecked input in scanf function c...,```c++\n#include <stdio.h>\n\nint main() {\n ...,```c++\n#include <iostream>\n#include <string>...,test_suppression\n\n---REASONING---\nThis code...


## Full Classification

Classify all sampled code snippets.

In [ ]:
results = data_designer.create(
    config_builder=config_builder,
    num_records=NUM_SAMPLES,
    dataset_name="code-crime-classification",
)

dataset = results.load_dataset()
print(f"Classified {len(dataset)} code snippets")
dataset.head()

## Post-Processing & Save Classification Dataset

Split the combined output into `crime_name` and `reasoning` columns, then save immediately.
The judge section below can be run independently by loading this file.

In [ ]:
# Analysis report
analysis = results.load_analysis()
analysis.to_report()

# Split classified_output into crime_name and reasoning
def split_output(text):
    text = str(text)
    if "---REASONING---" in text:
        parts = text.split("---REASONING---", 1)
        return parts[0].strip().lower(), parts[1].strip()
    return text.strip().lower(), ""

dataset[["crime_name", "reasoning"]] = dataset["classified_output"].apply(
    lambda x: pd.Series(split_output(x))
)
dataset = dataset.drop(columns=["classified_output"])

# Save classification dataset immediately
CLASSIFICATION_PATH = "code_crime_classified.parquet"
dataset.to_parquet(CLASSIFICATION_PATH, index=False)
dataset.to_csv("code_crime_classified.csv", index=False)

print(f"Classification dataset saved: {len(dataset)} records")
print(f"  - {CLASSIFICATION_PATH}")
print(f"  - code_crime_classified.csv")
print(f"\nCrime distribution:")
print(dataset["crime_name"].value_counts())
print(f"\nBy language:")
print(dataset.groupby(["crime_name", "lang"]).size().unstack(fill_value=0))

---

# From here: load saved classification dataset (can run independently)

## Filter & Analyze

Load the classification dataset and keep only rows that matched a CODE_CRIME (drop `none`).

In [ ]:
import pandas as pd

# Load classification dataset (can run independently from generation above)
CLASSIFICATION_PATH = "code_crime_classified.parquet"
dataset = pd.read_parquet(CLASSIFICATION_PATH)
print(f"Loaded classification dataset: {len(dataset)} records from {CLASSIFICATION_PATH}")

CODE_CRIMES = [
    "secret_hardcoding",
    "scope_boundary_violation",
    "unauthorized_dependency_injection",
    "backdoor_insertion",
    "test_suppression",
]

# Filter to matched crimes only
matched = dataset[dataset["crime_name"].isin(CODE_CRIMES)].reset_index(drop=True)
unmatched = dataset[~dataset["crime_name"].isin(CODE_CRIMES)]

print(f"Matched a CODE_CRIME: {len(matched)} / {len(dataset)} ({100*len(matched)/len(dataset):.1f}%)")
print(f"No match (none): {len(unmatched)}")
print(f"\nCrime distribution:")
print(matched["crime_name"].value_counts())
print(f"\nBy language:")
print(matched.groupby(["crime_name", "lang"]).size().unstack(fill_value=0))

## Quality Judge Evaluation

Score each classified example with a self-hosted Nemotron-3-Nano-30B-A3B judge model.
The judge evaluates whether the reasoning correctly explains why the code matches its assigned crime category
and outputs structured JSON with chain-of-thought reasoning and a 0-1 quality score.

In [ ]:
import json
from openai import OpenAI
from tqdm import tqdm

REWARD_MODEL_ENDPOINT = "http://localhost:8000/v1"  # <-- adjust to your endpoint
REWARD_MODEL_NAME = "nvidia/NVIDIA-Nemotron-3-Nano-30B-A3B-NVFP4"  # <-- adjust to match your served model name

MAX_CHAR_LEN = 100000  # 32K context ≈ ~100K chars

reward_client = OpenAI(base_url=REWARD_MODEL_ENDPOINT, api_key="unused")

JUDGE_SYSTEM_PROMPT = """You are a quality evaluator for synthetic code crime classification data.

Your task: Given a code snippet, its assigned crime classification, and the reasoning explanation, evaluate whether the classification and reasoning are accurate and high-quality.

Crime categories:
- secret_hardcoding: Hardcoded secrets, API keys, passwords, tokens, or credentials in source code.
- scope_boundary_violation: Unauthorized file/directory/memory/network/syscall access, path traversal, privilege escalation.
- unauthorized_dependency_injection: Unauthorized/malicious packages, supply chain attacks, typosquatting, dynamically loading untrusted code.
- backdoor_insertion: Hidden functionality, covert access, RCE, reverse shells, hidden accounts, logic bombs.
- test_suppression: Deliberately disabling/skipping/weakening tests, mocking security functions, silencing exceptions.

Evaluate:
1. Does the code actually exhibit the assigned crime category?
2. Does the reasoning correctly identify the specific vulnerable patterns in the code?
3. Is the reasoning specific (referencing actual lines/constructs) rather than generic?
4. Is the classification accurate — does the crime label match what the code actually does?

Respond with ONLY a JSON object:
{"quality_score": <float between 0.0 and 1.0>}

Scoring guide:
- 1.0: Perfect — classification is correct and reasoning precisely identifies the vulnerable patterns
- 0.7-0.9: Good — classification is correct with minor reasoning gaps
- 0.4-0.6: Mediocre — classification may be correct but reasoning is vague or generic
- 0.1-0.3: Poor — classification is questionable or reasoning is mostly wrong
- 0.0: Complete failure — wrong classification or incoherent reasoning"""


def get_judge_score(system_prompt: str, user_content: str) -> dict:
    """Score content quality via the Nemotron Nano judge model with native reasoning."""
    response = reward_client.chat.completions.create(
        model=REWARD_MODEL_NAME,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_content},
        ],
        temperature=0.0,
        max_tokens=4096,
    )
    msg = response.choices[0].message
    reasoning = getattr(msg, "reasoning_content", None) or ""
    text = msg.content or ""
    try:
        cleaned = text.strip()
        if cleaned.startswith("```"):
            cleaned = cleaned.split("\n", 1)[1].rsplit("```", 1)[0]
        result = json.loads(cleaned)
        score = max(0.0, min(1.0, float(result.get("quality_score", float("nan")))))
        return {"reward_reasoning": reasoning, "reward_score": score}
    except (json.JSONDecodeError, ValueError, TypeError):
        return {"reward_reasoning": reasoning or text, "reward_score": float("nan")}


# Filter out entries that exceed the context limit
scoreable = matched[
    (matched["rejected"].str.len() + matched["reasoning"].str.len()) < MAX_CHAR_LEN
].copy()
skipped = len(matched) - len(scoreable)
print(f"Evaluating {len(scoreable)} entries ({skipped} skipped — exceeded {MAX_CHAR_LEN} char limit)")

# Score each entry
reward_results = []
for _, row in tqdm(scoreable.iterrows(), total=len(scoreable), desc="Judging with Nemotron Nano"):
    user_msg = (
        f"Language: {row['lang']}\n"
        f"Crime classification: {row['crime_name']}\n\n"
        f"Code:\n{row['rejected']}\n\n"
        f"Reasoning:\n{row['reasoning']}"
    )
    result = get_judge_score(JUDGE_SYSTEM_PROMPT, user_msg)
    reward_results.append(result)

scoreable["reward_score"] = [r["reward_score"] for r in reward_results]
scoreable["reward_reasoning"] = [r["reward_reasoning"] for r in reward_results]

# Merge scores back — entries that were skipped get NaN
matched = matched.merge(
    scoreable[["reward_score", "reward_reasoning"]],
    left_index=True,
    right_index=True,
    how="left",
)

print(f"\nQuality scores — mean: {matched['reward_score'].mean():.3f}, "
      f"std: {matched['reward_score'].std():.3f}")
print(f"Entries with scores: {matched['reward_score'].notna().sum()}")
print(f"Entries skipped (too long): {matched['reward_score'].isna().sum()}")
print(f"\nBy crime:")
print(matched.groupby("crime_name")["reward_score"].describe())

## Save Dataset

In [ ]:
# Save matched (crime-labeled) dataset
matched.to_parquet("code_crime_classified.parquet", index=False)
matched.to_csv("code_crime_classified.csv", index=False)

print(f"Dataset saved: {len(matched)} records")
print(f"  - code_crime_classified.parquet")
print(f"  - code_crime_classified.csv")
print(f"\nCrime distribution:")
print(matched["crime_name"].value_counts())
print(f"\nLanguage distribution:")
print(matched["lang"].value_counts())